In [2]:
!nvidia-smi

Fri Jan 17 16:38:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:65:00.0 Off |                  N/A |
| 30%   47C    P8     6W / 250W |    164MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:B4:00.0 Off |                  N/A |
| 36%   57C    P0    40W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
test_scenes_X=["2011_09_26_drive_0117","2011_09_28_drive_0002","2011_09_26_drive_0052","2011_09_30_drive_0016","2011_09_26_drive_0059","2011_09_26_drive_0027","2011_09_26_drive_0020","2011_09_26_drive_0009","2011_09_26_drive_0013","2011_09_26_drive_0101","2011_09_26_drive_0046","2011_09_26_drive_0029","2011_09_26_drive_0064","2011_09_26_drive_0048","2011_10_03_drive_0027","2011_09_26_drive_0002","2011_09_26_drive_0036","2011_09_29_drive_0071","2011_10_03_drive_0047","2011_09_30_drive_0027","2011_09_26_drive_0086","2011_09_26_drive_0084","2011_09_26_drive_0096","2011_09_30_drive_0018","2011_09_26_drive_0106","2011_09_26_drive_0056","2011_09_26_drive_0023","2011_09_26_drive_0093"]
            

In [4]:
from __future__ import division
 
import numpy as np
import cv2
from tensorboardX import SummaryWriter
from path import Path
import scipy.misc,json
from pebble import ProcessPool
from collections import Counter
from tqdm import tqdm
import torch.utils.data as data
import numpy as np
from imageio import imread
import random,sys
import matplotlib.pyplot as plt
import torch.optim
import torch.nn as nn
from torch.autograd import Variable
from torchvision.transforms import transforms
import torchvision.utils as vutils
import torch.backends.cudnn as cudnn
device = 'cuda:1'
from PIL import Image
%matplotlib inline 
config_pathz=Path('cam_params.json')
from monodepth_utils import make_tensor
from monodepth_utils import tile
import torchvision.transforms as transforms
import time
import datetime
import logging
 
logger = logging.getLogger()

def setup_file_logger(log_file):
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter(' %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)

def log(message):
    #outputs to Jupyter console
    #outputs to file
    logger.info(message)
writer = SummaryWriter('run_flow_attn2/')
setup_file_logger('out6_sm.log')
config_pathz=Path('cam_params.json')


In [5]:
# import os
# for date in ['2011_09_26']:
#     drive_set = os.listdir("/home/adityacs16/kitti_unrect/"+ date + '/')
#     for dr in drive_set:
#         drive_dir = os.path.join("/home/adityacs16/kitti_unrect/", date, dr)
#         if os.path.isdir(drive_dir):
#             print(dr[:-5])
#             if dr[:-5] in self.test_scenes:
#                 continue
#             for cam in self.cam_ids:
#                 img_dir = os.path.join(drive_dir, 'image_' + cam, 'data')
#                 N = len(glob(img_dir + '/*.png'))
#                 for n in range(N):
#                     frame_id = '%.10d' % n
#                     all_frames.append(dr + ' ' + cam + ' ' + frame_id)

# u="2011_09_26_drive_0001_sync"
# u[:-5]

In [6]:
import torch.nn.functional as F
def upsample(x,k):
    """Upsample input tensor by a factor of 2
    """
    return F.interpolate(x, scale_factor=k, mode="bilinear")
upsample(torch.zeros(4,3,128,256),4).shape

/Data/tulikapcs16/anaconda3/envs/kaiser/lib/python3.7/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([4, 3, 512, 1024])

In [7]:
# from Resnet_Model_pixels import *
from LOSS_DEPTH import *
from Warp_functions import *
# from Warp_functions import ungrid_und_img as ungrid_und_img
from Pose_Model import Pose
# from MonoDepth_Model import *

In [8]:
# !pwd

In [9]:
config_pathz=Path('cam_params.json')
from path import Path
dump_root_eval = Path('/Data/tulikapcs16/kaiser/Dataset/test_dataset_unrect')
origin_test=Path('/Data/tulikapcs16/kaiser/Dataset/images/')   
dump_root_eval.mkdir_p()
def load_as_float(path):
    return imread(path).astype(np.float32)
class SequenceFolder_test(data.Dataset):
    def __init__(self, root ):

        self.root = Path(root)
        self.samples=((self.root.files()))
        self.samples.sort()
   

        
    def __len__(self):
        return len(self.samples)       

    def __getitem__(self, index):
        sample = self.samples[index]
        tgt_img = load_as_float(Path(sample))
        return str(sample),np.array(tgt_img)
        
test_data_loader=SequenceFolder_test(dump_root_eval)        
        


import argparse
import numpy as np
import cv2


def load_gt_disp_kitti(location):
    gt_disparities = []
    for i in range(200):
        try:

            disp = cv2.imread(Path(location) + str(i).zfill(6) + "_10.png", -1)
            disp = disp.astype(np.float32) / 256
            gt_disparities.append(disp)
        except:
            print("")
    return gt_disparities


def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    delta1 = (thresh < 1.25).mean()
    delta2 = (thresh < 1.25 ** 2).mean()
    delta3 = (thresh < 1.25 ** 3).mean()

    rmse = (gt - pred) ** 2
    rmse = np.sqrt(rmse.mean())

    rmse_log = (np.log(gt) - np.log(pred)) ** 2
    rmse_log = np.sqrt(rmse_log.mean())
#     print(gt.shape,pred.shape)
    abs_rel = np.mean(np.abs(gt - pred) / gt)

    sq_rel = np.mean(((gt - pred) ** 2) / gt)
    return abs_rel, sq_rel, rmse, rmse_log, delta1, delta2, delta3


def convert_disps_to_depths_kitti(gt_disparities, predicted_depths):
    gt_depths = []
    pred_depths = []
    pred_depths_resized = []

    focal_length = 721.5377
    baseline = 0.54

    for i in range(len(gt_disparities)):
        gt_disp = gt_disparities[i]
        height, width = 512, 1392

        pred_depth =1/predicted_depths[i]
        pred_depth = cv2.resize(pred_depth, (width, height), interpolation=cv2.INTER_LINEAR)


        pred_depths_resized.append(pred_depth)
        mask = gt_disp > 0

        gt_depth = focal_length * baseline / (gt_disp + (1.0 - mask))
        pred_depths.append(pred_depth)
        gt_depths.append(gt_depth)
    return gt_depths, pred_depths, pred_depths_resized





# print(len(gt_disparities))
# print(disp.shape)


def eval(model,test_data_loader):
    
    disparity = np.zeros((len(test_data_loader),256, 512), dtype=np.float32)
    u=0
    for _,img in test_data_loader:
        inputz=torch.tensor(img).reshape(1,256,512,3).permute(0,3,1,2)
        inp_image=inputz.to(device)
        inp_image=inp_image/255
        disp_map=model(inp_image)
        disp_last_layer=disp_map[0][0][0]
        disparity[u]=np.array(disp_last_layer.cpu().data)
        u=u+1
        
#         print(u)
    np.save( 'xx7.npy', disparity)
    disp = np.load('xx7.npy')
    num_samples = (disp.shape[0])
    gt_disparities = load_gt_disp_kitti('/Data/tulikapcs16/kaiser/Dataset/disp_GT/')
    gt_depths, predicted_depths, pred_depths_resized = convert_disps_to_depths_kitti(gt_disparities, disp)
    num_samples=142
    rms = np.zeros(num_samples, np.float32)
    log_rms = np.zeros(num_samples, np.float32)
    abs_rel = np.zeros(num_samples, np.float32)
    sq_rel = np.zeros(num_samples, np.float32)
    d1_all = np.zeros(num_samples, np.float32)
    delta1 = np.zeros(num_samples, np.float32)
    delta2 = np.zeros(num_samples, np.float32)
    delta3 = np.zeros(num_samples, np.float32)

    min_depth=1e-3
    max_depth=80
    K_02 = np.array([9.597910e+02, 0.000000e+00, 6.960217e+02,
                     0.000000e+00, 9.569251e+02, 2.241806e+02,
                     0.000000e+00, 0.000000e+00, 1.000000e+00]).reshape(3, 3)

    D = np.array([-3.691481e-01, 1.968681e-01, 1.353473e-03, 5.677587e-04, -6.770705e-02])

    R_rect_02 = np.array([9.998817e-01, 1.511453e-02, -2.841595e-03,
                          -1.511724e-02, 9.998853e-01, -9.338510e-04,
                          2.827154e-03, 9.766976e-04, 9.999955e-01]).reshape(3, 3)

    P_rect_02 = np.array([7.215377e+02, 0.000000e+00, 6.095593e+02,
                          0.000000e+00, 7.215377e+02, 1.728540e+02,
                          0.000000e+00, 0.000000e+00, 1.000000e+00]).reshape(3, 3)

    map1, map2 = cv2.initUndistortRectifyMap(K_02, D, R_rect_02, P_rect_02, (1242, 375), cv2.CV_32FC1)

    v=0
    jv=0
    mv=[]
    ff=[]
    zx_abs=zx_sq=zx_rms=zx_log=zx_rms=zx_sq=zx_d1=zx_delta1=zx_delta2=zx_delta3=0
    for i in range(142):

    #     if i   in mylist:
    #         continue


        gt_depth = gt_depths[i]
        pred_depth = predicted_depths[i]

    #     pred_depth[pred_depth < min_depth] = min_depth
    #     pred_depth[pred_depth > max_depth] = max_depth

        gt_disp = gt_disparities[i]
        mask = gt_disp > 0
        mask = np.logical_and(gt_depth > min_depth,
                              gt_depth < max_depth)
        pred_d = pred_depths_resized[i]

        pred_depth = cv2.remap(pred_d, map1, map2, cv2.INTER_LINEAR)
    #     pred_depth=pred_disp

        height, width = gt_disp.shape
        pred_depth = cv2.resize(pred_depth, (width, height), interpolation=cv2.INTER_LINEAR) 
    #     u=np.std(gt_depth[mask])/np.std(pred_depth[mask])

        pred_depth=pred_depth*(np.median(gt_depth[mask])/np.median(pred_depth[mask]))
    #     pred_depth[pred_depth < min_depth] = min_depth
    #     pred_depth[pred_depth > max_depth] = max_depth


        depth_diff = np.abs(gt_depth[mask] - pred_depth[mask])
        bad_pixels = np.logical_and(depth_diff >= 3, (depth_diff / gt_depth[mask]) >= 0.05)
        d1_all[i] = 100.0 * bad_pixels.sum() / mask.sum()

        abs_rel[i], sq_rel[i], rms[i], log_rms[i], delta1[i], delta2[i], delta3[i] = \
            compute_errors(gt_depth[mask], pred_depth[mask])
#         print(i,abs_rel[i],rms[i])
        if abs_rel[i]>=0.2:
            jv=jv+1
            ff.append(abs_rel[i])
            mv.append(i)
        else:
            zx_abs=zx_abs+abs_rel[i]
            zx_sq=zx_sq+sq_rel[i]
            zx_rms=zx_rms+rms[i]
            zx_log=zx_log+log_rms[i]
            zx_d1=zx_d1+d1_all[i]
            zx_delta1=zx_delta1+delta1[i]
            zx_delta2=zx_delta2+delta2[i]
            zx_delta3=zx_delta3+delta3[i]
    print("{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(),
                                                                                                  sq_rel.mean(),
                                                                                                  rms.mean(),
                                                                                                  log_rms.mean(),
                                                                                                  d1_all.mean(),
                                                                                                  delta1.mean(),
                                                                                                  delta2.mean(),
                                                                                                  delta3.mean())
         )
    return(abs_rel.mean(),sq_rel.mean(),rms.mean())


    
# eval(model,test_data_loader) 
    

In [10]:
# eigen_split
import skimage
from skimage.transform import resize as fresize
import os
def test(model,device):
    min_depth=1e-3
    max_depth=80
    dataset_dir='/Data/tulikapcs16/kaiser/Dataset/kitti_unrect/'

    test_file='eigen_unrect_input_gt_paths.txt'
    gt_root='/Data/tulikapcs16/kaiser/Dataset/kitti_gt_improved/'
    test_files = [os.path.join(dataset_dir, line.rstrip('\n').split(' ')[0]) for line in
                  open(test_file)]
    gt_files = [os.path.join(gt_root, line.rstrip('\n').split(' ')[1]) for line in
                open(test_file)]


    print(f"Total number of test images: {len(test_files)}")


    unrect_size = (1392, 512)
    # rect_size = (1242, 375)

    pred_depths = []
    gt_depths = []


#     print("\n=> Running test files")
    for i, (image_path, gt_path) in (enumerate(zip(test_files, gt_files))):
        image = load_as_float(image_path)
        image = fresize(image,[256,512])
#         print(i)
        gt = Image.open(gt_path)

        tensor_image = torch.tensor(image).permute(2,0,1).unsqueeze(0).to(device).float()

#         print(tensor_image.shape)
        output = model(tensor_image/255)

        depth = 1/output[0][0][0].detach().cpu().numpy()

        upsample_pred_depth = cv2.resize(depth, unrect_size, interpolation=cv2.INTER_LINEAR)
        pred_depths.append(upsample_pred_depth)
        gt_depths.append(np.array(gt).astype(np.float32) / 256)
#         print(i)



    if 1==1:
        print("\nEvaluation results...")

        num_samples = len(pred_depths)
        rms = np.zeros(num_samples, np.float32)
        log_rms = np.zeros(num_samples, np.float32)
        abs_rel = np.zeros(num_samples, np.float32)
        sq_rel = np.zeros(num_samples, np.float32)
        d1_all = np.zeros(num_samples, np.float32)
        delta1 = np.zeros(num_samples, np.float32)
        delta2 = np.zeros(num_samples, np.float32)
        delta3 = np.zeros(num_samples, np.float32)

        K_02 = np.array([9.597910e+02, 0.000000e+00, 6.960217e+02,
                         0.000000e+00, 9.569251e+02, 2.241806e+02,
                         0.000000e+00, 0.000000e+00, 1.000000e+00]).reshape(3, 3)

        D = np.array([-3.691481e-01, 1.968681e-01, 1.353473e-03, 5.677587e-04, -6.770705e-02])

        R_rect_02 = np.array([9.998817e-01, 1.511453e-02, -2.841595e-03,
                              -1.511724e-02, 9.998853e-01, -9.338510e-04,
                              2.827154e-03, 9.766976e-04, 9.999955e-01]).reshape(3, 3)

        P_rect_02 = np.array([7.215377e+02, 0.000000e+00, 6.095593e+02,
                              0.000000e+00, 7.215377e+02, 1.728540e+02,
                              0.000000e+00, 0.000000e+00, 1.000000e+00]).reshape(3, 3)

        map1, map2 = cv2.initUndistortRectifyMap(K_02, D, R_rect_02, P_rect_02, (1242, 375), cv2.CV_32FC1)

        for i in range(num_samples):
            gt_depth = gt_depths[i]
            pred_depth = pred_depths[i]

            if 1==1:
                gt_height, gt_width = gt_depth.shape[:2]
                mask = np.logical_and(gt_depth > min_depth, gt_depth < max_depth)

                crop = np.array([0.40810811 * gt_height, 0.99189189 * gt_height,
                                 0.03594771 * gt_width, 0.96405229 * gt_width]).astype(np.int32)
                crop_mask = np.zeros(mask.shape)
                crop_mask[crop[0]:crop[1], crop[2]:crop[3]] = 1
                mask = np.logical_and(mask, crop_mask)




            pred_depth[pred_depth < min_depth] = min_depth
            pred_depth[pred_depth > max_depth] = max_depth

            pred_depth = cv2.remap(pred_depth, map1, map2, cv2.INTER_LINEAR)
            height, width = gt_depth.shape
            pred_depth = cv2.resize(pred_depth, (width, height), interpolation=cv2.INTER_LINEAR)

            gt_depth = gt_depth[mask]
            pred_depth = pred_depth[mask]

            if 1==1:
                ratio = np.median(gt_depth) / np.median(pred_depth)
                pred_depth *= ratio

            depth_diff = np.abs(gt_depth - pred_depth)
            bad_pixels = np.logical_and(depth_diff >= 3, (depth_diff / gt_depth) >= 0.05)
            d1_all[i] = 100.0 * bad_pixels.sum() / mask.sum()

            abs_rel[i], sq_rel[i], rms[i], log_rms[i], delta1[i], delta2[i], delta3[i] = \
                 compute_errors(gt_depth, pred_depth)
        print("eigen_split")
        print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('abs_rel', 'sq_rel', 'rms',
                                                                                      'log_rms',
                                                                                      'd1_all', 'a1', 'a2', 'a3'))
        print(
            "{:10.4f}, {:10.4f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}, {:10.3f}".format(abs_rel.mean(),
                                                                                                    sq_rel.mean(),
                                                                                                    rms.mean(),
                                                                                                    log_rms.mean(),
                                                                                                    d1_all.mean(),
                                                                                                    delta1.mean(),
                                                                                                    delta2.mean(),
                                                                                                    delta3.mean()))
        return(abs_rel.mean())
        
# model=ResnetModel(3,'resnet50',pretrained=True) 
# # model.load_state_dict(torch.load('Res18ATTm1.pt',map_location='cpu'))
# test(model,'cpu')

In [11]:
# import models
# pretrained='/home/adityacs16/WARP_MOTION/FlowNetPytorch-master/flownets_EPE1.951.pth'
# network_data = torch.load(pretrained,map_location='cpu') 
# print("=> using pre-trained model '{}'".format(network_data['arch']))
# model_flow = models.__dict__[network_data['arch']](network_data).to(device)

In [12]:
# from path import Path
# def load_as_float(path):
#     return imread(path).astype(np.float32)
# class SequenceFolder2(data.Dataset):
#     def __init__(self, root ):

#         self.root = Path(root)
#         self.samples=((self.root.files()))
#         self.samples.sort()
   

        
#     def __len__(self):
#         return len(self.samples)       

#     def __getitem__(self, index):
#         sample = self.samples[index]
#         tgt_img = load_as_float(Path(sample))
#         return np.transpose(np.array(tgt_img),(2,0,1))

# dump_root2 = Path('/home/adityacs16/WARP_MOTION/test_dataset_unrect')
# test_data_loader=SequenceFolder2(dump_root2)

# test_loader = torch.utils.data.DataLoader(
#     test_data_loader, batch_size=2, shuffle=False,
#     num_workers=1, pin_memory=False)


In [13]:
def read_calib_file(path):
    float_chars = set("0123456789.e+- ")
    data = {}
    with open(path, 'r') as f:
        for line in f.readlines():
            key, value = line.split(':', 1)
            value = value.strip()
            data[key] = value
            if float_chars.issuperset(value):
                # try to cast to float array
                try:
                    data[key] = np.array(list(map(float, value.split(' '))))
                except ValueError:
                    # casting error: data[key] already eq. value, so pass
                    pass

    return(data)

In [14]:
dump_root = Path('/Data/tulikapcs16/kaiser/Dataset/testrun_TOTAL/')
with open('kaiser_banned.txt') as f:
    banned_content = f.readlines()
banned_content = [x.strip() for x in banned_content] 
print(banned_content)
print(len(banned_content))

['2011_09_29_drive_0071_extract_02/00000000.png', '2011_09_29_drive_0071_extract_02/00000001.png', '2011_09_29_drive_0071_extract_02/00000002.png', '2011_09_29_drive_0071_extract_02/00000003.png', '2011_09_29_drive_0071_extract_02/00000004.png', '2011_09_29_drive_0071_extract_02/00000005.png', '2011_09_29_drive_0071_extract_02/00000006.png', '2011_09_29_drive_0071_extract_02/00000007.png', '2011_09_29_drive_0071_extract_02/00000008.png', '2011_09_29_drive_0071_extract_02/00000009.png', '2011_09_29_drive_0071_extract_02/00000010.png', '2011_09_29_drive_0071_extract_02/00000011.png', '2011_09_29_drive_0071_extract_02/00000012.png', '2011_09_29_drive_0071_extract_02/00000013.png', '2011_09_29_drive_0071_extract_02/00000014.png', '2011_09_29_drive_0071_extract_02/00000015.png', '2011_09_29_drive_0071_extract_02/00000016.png', '2011_09_29_drive_0071_extract_02/00000017.png', '2011_09_29_drive_0071_extract_02/00000018.png', '2011_09_29_drive_0071_extract_02/00000019.png', '2011_09_29_drive_0

In [15]:
input_transform = transforms.Compose([
    transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
    transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
])


def load_as_float(path):
    return imread(path).astype(np.float32)
class SequenceFolder(data.Dataset):
    def __init__(self, root, seed=None, train=True, sequence_length=3, transform=None, target_transform=None):
        np.random.seed(seed)
        random.seed(seed)
        self.root = Path(root)

        self.transformz = transforms.Compose([transforms.ToTensor()])  
        scene_list_path = dump_root/'train.txt'
        self.scenes = []
        for folder in open(scene_list_path):
            if(folder[-2]=='3'):
                continue
#             print(folder)
            p0='/testrun_TOTAL/'
            

            self.scenes.append(self.root/folder[:-1])
        self.transform = transform
        self.crawl_folders(sequence_length)
    def crawl_folders(self, sequence_length):
        sequence_set = []
        demi_length = (sequence_length-1)//2
        shifts = list(range(-demi_length, demi_length + 1))
        shifts.pop(demi_length)
        for scene in self.scenes:
            
            if scene[-32:][:-11] in test_scenes_X:
                print(scene[-32:][:-11])
                print("WRONG scene")
                continue
            intrinsics = np.genfromtxt(scene/'cam.txt').astype(np.float32).reshape((3, 3))
            imgs = sorted((scene).files('*.png'))
            if len(imgs) < sequence_length:
                continue
            for i in range(demi_length, len(imgs)-demi_length):
                sample = {'intrinsics': intrinsics, 'tgt': imgs[i], 'ref_imgs': []}
                jpgfile = Image.open(imgs[i])
#                 print(sample['tgt'][-14])
                y_n=1
#                 print(jpgfile.size)
                for j in shifts:
#                     sample['ref_imgs'].append(imgs[i+j])
                    img1=torch.tensor(load_as_float(imgs[i])).permute(2,0,1) #t-1
                    img2=torch.tensor(load_as_float(imgs[i+j])).permute(2,0,1) #t
                    img1 = input_transform(img1)
                    img2 = input_transform(img2)
#                     print(imgs[i][14:])
                    if (imgs[i][-45:] in banned_content )or (imgs[i+j][-45:] in banned_content):
                        y_n=0
                        print("its banned bois")
                    else:
                        
                        print("its not banned")
                    input_var = torch.cat([img1, img2]).unsqueeze(0)
#                     xzz=model_flow(input_var.to(device))
#                     u=torch.mean(torch.abs(xzz[0][0][0]).cpu().data+torch.abs(xzz[0][0][1]).cpu().data)
#                     u2=torch.sum(torch.abs(xzz[0][0][0]).cpu().data+torch.abs(xzz[0][0][1]).cpu().data>1)
                    sample['ref_imgs'].append(imgs[i+j])
#                     if(u<=0.5 or u2<50):
#                         y_n=0
                if(y_n==0):
                    continue
                sequence_set.append(sample)
#                 if(len(sequence_set)>1000):
#                     break
#             if(len(sequence_set)>1000):
#                 break
        random.shuffle(sequence_set)
        self.samples = sequence_set

        
    def __len__(self):
        return len(self.samples)       

    def __getitem__(self, index):
        sample = self.samples[index]
        tgt_img = load_as_float(sample['tgt'])
        ref_imgs = [load_as_float(ref_img) for ref_img in sample['ref_imgs']]
#         print(tgt_img.shape)
#         print (len(ref_imgs))
        
        intrinsics = np.copy(sample['intrinsics'])
        return np.array(tgt_img), np.array(ref_imgs)
train_set = SequenceFolder(
    root=dump_root,
    transform=None,
    seed=1,
    train=True,
    sequence_length=3
)
batch_size=4
workers=4
sequence_length=3
photo_loss_weight=1
smooth_loss_weight=0.1

2011_09_26_drive_0020
WRONG scene
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned 

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

...

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its banned bois
its bann

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not banned
its not ba

In [16]:
!pwd

/Data/tulikapcs16/kaiser/Dataset/KITTI


In [18]:
print(len(train_set))

17961


In [19]:
train_loader_2 = torch.utils.data.DataLoader(
    train_set, batch_size=4, shuffle=True,
    num_workers=1, pin_memory=False,drop_last=True)


    
epoch_size = len(train_set)
print(epoch_size)
q=[]
test_tgt=[]
test_ref=[]
for i, (tgt_img, ref_imgs) in enumerate(train_loader_2):
#     print(tgt_img.shape)
    q.append(np.array(tgt_img[0]))
    q.append(np.array(ref_imgs[0][0]))
    q.append(np.array(ref_imgs[0][1]))
    test_tgt.append(tgt_img)
    test_ref.append(ref_imgs)
    break
# !nvidia-smi

17961


In [20]:

# coding: utf-8

# In[2]:


import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


#MODELS
import importlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim,activation):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        self.activation = activation
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out,attention    


class conv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, stride):
        super(conv, self).__init__()
        self.kernel_size = kernel_size
        self.conv_base = nn.Conv2d(num_in_layers, num_out_layers, kernel_size=kernel_size, stride=stride)
        self.normalize = nn.BatchNorm2d(num_out_layers)

    def forward(self, x):
        p = int(np.floor((self.kernel_size - 1) / 2))
        p2d = (p, p, p, p)
        x = self.conv_base(F.pad(x, p2d))
        x = self.normalize(x)
        return F.elu(x, inplace=True)


class convblock(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size):
        super(convblock, self).__init__()
        self.conv1 = conv(num_in_layers, num_out_layers, kernel_size, 1)
        self.conv2 = conv(num_out_layers, num_out_layers, kernel_size, 2)

    def forward(self, x):
        x = self.conv1(x)
        return self.conv2(x)


class maxpool(nn.Module):
    def __init__(self, kernel_size):
        super(maxpool, self).__init__()
        self.kernel_size = kernel_size

    def forward(self, x):
        p = int(np.floor((self.kernel_size - 1) / 2))
        p2d = (p, p, p, p)
        return F.max_pool2d(F.pad(x, p2d), self.kernel_size, stride=2)


class resconv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, stride):
        super(resconv, self).__init__()
        self.num_out_layers = num_out_layers
        self.stride = stride
        self.conv1 = conv(num_in_layers, num_out_layers, 1, 1)
        self.conv2 = conv(num_out_layers, num_out_layers, 3, stride)
        self.conv3 = nn.Conv2d(num_out_layers, 4 * num_out_layers, kernel_size=1, stride=1)
        self.conv4 = nn.Conv2d(num_in_layers, 4 * num_out_layers, kernel_size=1, stride=stride)
        self.normalize = nn.BatchNorm2d(4 * num_out_layers)

    def forward(self, x):
        # do_proj = x.size()[1] != self.num_out_layers or self.stride == 2
        do_proj = True
        shortcut = []
        x_out = self.conv1(x)
        x_out = self.conv2(x_out)
        x_out = self.conv3(x_out)
        if do_proj:
            shortcut = self.conv4(x)
        else:
            shortcut = x
        return F.elu(self.normalize(x_out + shortcut), inplace=True)


class resconv_basic(nn.Module):
    # for resnet18
    def __init__(self, num_in_layers, num_out_layers, stride):
        super(resconv_basic, self).__init__()
        self.num_out_layers = num_out_layers
        self.stride = stride
        self.conv1 = conv(num_in_layers, num_out_layers, 3, stride)
        self.conv2 = conv(num_out_layers, num_out_layers, 3, 1)
        self.conv3 = nn.Conv2d(num_in_layers, num_out_layers, kernel_size=1, stride=stride)
        self.normalize = nn.BatchNorm2d(num_out_layers)

    def forward(self, x):
        # do_proj = x.size()[1] != self.num_out_layers or self.stride == 2
        do_proj = True
        shortcut = []
        x_out = self.conv1(x)
        x_out = self.conv2(x_out)
        if do_proj:
            shortcut = self.conv3(x)
        else:
            shortcut = x
        return F.elu(self.normalize(x_out + shortcut), inplace=True)


def resblock(num_in_layers, num_out_layers, num_blocks, stride):
    layers = []
    layers.append(resconv(num_in_layers, num_out_layers, stride))
    for i in range(1, num_blocks - 1):
        layers.append(resconv(4 * num_out_layers, num_out_layers, 1))
    layers.append(resconv(4 * num_out_layers, num_out_layers, 1))
    return nn.Sequential(*layers)


def resblock_basic(num_in_layers, num_out_layers, num_blocks, stride):
    layers = []
    layers.append(resconv_basic(num_in_layers, num_out_layers, stride))
    for i in range(1, num_blocks):
        layers.append(resconv_basic(num_out_layers, num_out_layers, 1))
    return nn.Sequential(*layers)


class upconv(nn.Module):
    def __init__(self, num_in_layers, num_out_layers, kernel_size, scale):
        super(upconv, self).__init__()
        self.scale = scale
        self.conv1 = conv(num_in_layers, num_out_layers, kernel_size, 1)

    def forward(self, x):
        x = nn.functional.interpolate(x, scale_factor=self.scale, mode="bilinear")
        return self.conv1(x)


class get_disp(nn.Module):
    def __init__(self, num_in_layers):
        super(get_disp, self).__init__()
        self.conv1 = nn.Conv2d(num_in_layers, 1, kernel_size=3, stride=1)
        self.elu = torch.nn.ELU()
        self.sigmoid=nn.Sigmoid()
    def forward(self, x):
        p = 1
        p2d = (p, p, p, p)
        x = self.conv1(F.pad(x, p2d))
        
        # Sigmoid layer for disparity 0.3 / (1+ e^-x)
        # Model inverse sigmoid layer to obtain depth 3.33 * (1+ e^-x)
        return (10 * (self.sigmoid(x))) + 0.01





def class_for_name(module_name, class_name):
    # load the module, will raise ImportError if module cannot be loaded
    m = importlib.import_module(module_name)
    # get the class, will raise AttributeError if class cannot be found
    return getattr(m, class_name)


class ResnetModel(nn.Module):
    def __init__(self, num_in_layers, encoder='resnet18', pretrained=False):
        super(ResnetModel, self).__init__()
        assert encoder in ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'], "Incorrect encoder type"
        if encoder in ['resnet18', 'resnet34']:
            filters = [64, 128, 256, 512]
        else:
            filters = [256, 512, 1024, 2048]
        resnet = class_for_name("torchvision.models", encoder)(pretrained=pretrained)
        if num_in_layers != 3:  # Number of input channels
            self.firstconv = nn.Conv2d(num_in_layers, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        else:
            self.firstconv = resnet.conv1  # H/2
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool  # H/4

        # encoder
        self.encoder1 = resnet.layer1  # H/4
        self.encoder2 = resnet.layer2  # H/8
        self.encoder3 = resnet.layer3  # H/16
        self.encoder4 = resnet.layer4  # H/32

        # decoder
        self.upconv6 = upconv(filters[3], 512, 3, 2)
        self.iconv6 = conv(filters[2] + 512, 512, 3, 1)

        self.upconv5 = upconv(512, 256, 3, 2)
        self.iconv5 = conv(filters[1] + 256, 256, 3, 1)

        self.upconv4 = upconv(256, 128, 3, 2)
        self.iconv4 = conv(filters[0] + 128, 128, 3, 1)
        self.depth4_layer = get_disp(128)

        self.upconv3 = upconv(128, 64, 3, 1)  #
        self.iconv3 = conv(64 + 64 + 1, 64, 3, 1)
        self.depth3_layer = get_disp(64)

        self.upconv2 = upconv(64, 32, 3, 2)
        self.iconv2 = conv(64 + 32 + 1, 32, 3, 1)
        self.depth2_layer = get_disp(32)

        self.upconv1 = upconv(32, 16, 3, 2)
        self.iconv1 = conv(16 + 1, 16, 3, 1)
        self.depth1_layer = get_disp(16)
        
        
        
        
        self.att1=Self_Attn(64,'relu')
        self.att2=Self_Attn(128,'relu')
        self.att3=Self_Attn(256,'relu')
        self.att4=Self_Attn(512,'relu')

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                
            
                

    def forward(self, x):
        # encoder
        x_first_conv = self.firstconv(x)
        x = self.firstbn(x_first_conv)
        x = self.firstrelu(x)
        x_pool1 = self.firstmaxpool(x)
        x1 = self.encoder1(x_pool1)
        x1=self.att1(x1)[0]
#         print("ora")
        x2 = self.encoder2(x1)
        x2=self.att2(x2)[0]
        x3 = self.encoder3(x2)
        x3=self.att3(x3)[0]
#         print(x1.shape,x2.shape,x3.shape)
        x4 = self.encoder4(x3)
        x4=self.att4(x4)[0]
#         print(x4.shape)
        # skips
        skip1 = x_first_conv
        skip2 = x_pool1
        skip3 = x1
        skip4 = x2
        skip5 = x3

        # decoder
        
        upconv6 = self.upconv6(x4)
        concat6 = torch.cat((upconv6, skip5), 1)
        iconv6 = self.iconv6(concat6)

        upconv5 = self.upconv5(iconv6)
        concat5 = torch.cat((upconv5, skip4), 1)
        iconv5 = self.iconv5(concat5)

        upconv4 = self.upconv4(iconv5)
        concat4 = torch.cat((upconv4, skip3), 1)
        iconv4 = self.iconv4(concat4)
        self.depth4 = self.depth4_layer(iconv4)
        self.udepth4 = F.interpolate(self.depth4, scale_factor=1, mode="bilinear")
        self.depth4 = F.interpolate(self.depth4, scale_factor=0.5, mode="bilinear")
        
        upconv3 = self.upconv3(iconv4)
        concat3 = torch.cat((upconv3, skip2, self.udepth4), 1)
        iconv3 = self.iconv3(concat3)
        self.depth3 = self.depth3_layer(iconv3)
        self.udepth3 = F.interpolate(self.depth3, scale_factor=2, mode="bilinear")

        upconv2 = self.upconv2(iconv3)
        concat2 = torch.cat((upconv2, skip1, self.udepth3), 1)
        iconv2 = self.iconv2(concat2)
        self.depth2 = self.depth2_layer(iconv2)
        self.udepth2 = F.interpolate(self.depth2, scale_factor=2, mode="bilinear")

        upconv1 = self.upconv1(iconv2)
        concat1 = torch.cat((upconv1, self.udepth2), 1)
        iconv1 = self.iconv1(concat1)
        self.depth1 = self.depth1_layer(iconv1)
        return [self.depth1, upsample(self.depth2,2), upsample(self.depth3,4), upsample(self.depth4,8),[self.depth1, self.depth2, self.depth3, self.depth4]]

 
# device='cpu'    
model=ResnetModel(3,'resnet18').to('cpu') 
model(torch.zeros(4,3,256,512).to('cpu') )


/Data/tulikapcs16/anaconda3/envs/kaiser/lib/python3.7/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


[tensor([[[[7.0980, 9.1367, 7.6992,  ..., 1.8107, 1.2275, 2.8161],
           [1.9502, 3.5006, 0.0939,  ..., 0.2508, 0.3759, 7.3168],
           [4.1858, 0.0821, 0.0372,  ..., 0.7436, 0.2573, 6.8277],
           ...,
           [7.6307, 0.1244, 3.8423,  ..., 0.0577, 0.1419, 0.8516],
           [3.9648, 0.0350, 0.0415,  ..., 0.0467, 0.2018, 0.0736],
           [3.6943, 6.6171, 1.7760,  ..., 0.2171, 5.2745, 0.4268]]],
 
 
         [[[7.0980, 9.1367, 7.6992,  ..., 1.8107, 1.2275, 2.8161],
           [1.9502, 3.5006, 0.0939,  ..., 0.2508, 0.3759, 7.3168],
           [4.1858, 0.0821, 0.0372,  ..., 0.7436, 0.2573, 6.8277],
           ...,
           [7.6307, 0.1244, 3.8423,  ..., 0.0577, 0.1419, 0.8516],
           [3.9648, 0.0350, 0.0415,  ..., 0.0467, 0.2018, 0.0736],
           [3.6943, 6.6171, 1.7760,  ..., 0.2171, 5.2745, 0.4268]]],
 
 
         [[[7.0980, 9.1367, 7.6992,  ..., 1.8107, 1.2275, 2.8161],
           [1.9502, 3.5006, 0.0939,  ..., 0.2508, 0.3759, 7.3168],
           [4.1858

In [21]:
!nvidia-smi

Fri Jan 17 17:16:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:65:00.0 Off |                  N/A |
| 30%   40C    P8     7W / 250W |    164MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:B4:00.0 Off |                  N/A |
| 36%   45C    P0    35W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [22]:
device='cuda:0'

model1=ResnetModel(3,'resnet18',pretrained=True).to(device)
model2=Pose().to(device)

In [23]:
# model1.load_state_dict(torch.load('eigenRes50ATTm1.pt'))

In [24]:
# model2.load_state_dict(torch.load('eigenRes50ATTm2.pt'))
# 1+1

In [25]:
optim_params = [

      {'params': model1.parameters(), 'lr':0.0001},
      {'params': model2.parameters(), 'lr':0.0001}
]
optimizer = torch.optim.Adam(optim_params)


In [26]:
!nvidia-smi

Fri Jan 17 17:16:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:65:00.0 Off |                  N/A |
| 30%   44C    P2    61W / 250W |   1041MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:B4:00.0 Off |                  N/A |
| 29%   45C    P8    21W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:
epoch=520
jjjj=0
maxx=200
sigm=torch.nn.Sigmoid()
for xxxt in range(epoch):
    ij=0+xxxt
    loss_epoch=0
    if xxxt%2==0:
        print("saving")
#         log("6saving")
#         torch.save(model1.state_dict(),'DepthXFlownet-6/'+str(xxxt)+'flow6_expt1.pt')
# #         torch.save(model1_encoder.state_dict(), 'DepthXFlownet-6/'+str(xxxt)+'flow6_distm1en_epoch.pt')
#         torch.save(model2.state_dict(), 'DepthXFlownet-6/'+str(xxxt)+'flow6_expt2.pt')
        try:
          
            print("saved")
        except:
            print("sorry no memory")


        print("6saving finished") 
        log("6saving finished")
     
    for i, (tgt_img, ref_imgs) in enumerate(train_loader_2):



        torch.backends.cudnn.enabled =True
        ztgt_img=tgt_img.permute(0,3,1,2)        

        ztgt_img=ztgt_img.to(device)
        ztgt_img=ztgt_img/255

        ref_img1=ref_imgs[:,0,:,:,:].to(device)
        ref_img2=ref_imgs[:,1,:,:,:].to(device)
        ref_img1=ref_img1.permute(0,3,1,2)
        ref_img2=ref_img2.permute(0,3,1,2)

        ref_img1=ref_img1/255
        ref_img2=ref_img2/255
        gz=meshgrid(256,512)

        before_un_ref1=ref_img1
        before_un_ref2=ref_img2
        before_tgt=ztgt_img
        zcamera_matrix,znew_camera_matrix,zdistort,zrotation_rect=get_other_parameters(config_pathz,'2',[512,256])

        ref_img1=ungrid_und_img(ref_img1,zcamera_matrix.cpu().numpy(),znew_camera_matrix.cpu().numpy(),zdistort,zrotation_rect.cpu().numpy(),device)
        ref_img2=ungrid_und_img(ref_img2,zcamera_matrix.cpu().numpy(),znew_camera_matrix.cpu().numpy(),zdistort,zrotation_rect.cpu().numpy(),device)
        ztgt_img=ungrid_und_img(ztgt_img,zcamera_matrix.cpu().numpy(),znew_camera_matrix.cpu().numpy(),zdistort,zrotation_rect.cpu().numpy(),device)

#         pyramid_disp=model1_decoder(model1_encoder(before_tgt))
        pyramid_disp=model1(before_tgt)
#         print(pyramid_disp[0].shape)
        pose_estimate=model2(ref_img1,[ ztgt_img,ref_img2])
        pose_1=pose_estimate[:,0,:]
        pose_2=-pose_estimate[:,1,:]


        warped_image1_pyramid,mask1_pyramid=inverse_warp_pyramid3(before_un_ref1,pyramid_disp,256,512,'2',pose_1,device,config_pathz,0)

        warped_image2_pyramid,mask2_pyramid=inverse_warp_pyramid3(before_un_ref2,pyramid_disp,256,512,'2',pose_2,device,config_pathz,1)
        zcamera_matrix=zcamera_matrix.to(device)
        zrotation_rect=zrotation_rect.to(device)
        znew_camera_matrix=znew_camera_matrix.to(device)
  
        

#         dp=1/pyramid_disp[0]
        loss_batch,id_mp=loss_pyramid7_nomask(warped_image1_pyramid,warped_image2_pyramid,before_tgt,before_un_ref1,before_un_ref2,pyramid_disp[4],mask1_pyramid,mask2_pyramid,device)
 
        loss_batches=loss_batch/4
        
#         loss1=loss(before_tgte,warped_image1_pyramid[0])
#         loss2=loss(before_tgt,warped_image2_pyramid[0])
#         loss_batch=loss1+loss2
        xxx=before_tgt
        optimizer.zero_grad()  
        loss_batches.backward()
        optimizer.step()
        loss_epoch=loss_epoch+loss_batches


#         writer.add_image(f'{xxxt}_{i}',  vutils.make_grid(before_tgt),jjjj)
#         writer.add_image(f'warp1_{xxxt}_{i}', vutils.make_grid(warped_image1_pyramid[0]),jjjj)
#         writer.add_image(f'warp2_{xxxt}_{i}', vutils.make_grid(warped_image2_pyramid[0]),jjjj)
#         writer.add_image(f'depth_{xxxt}_{i}', vutils.make_grid(pyramid_depth[0]),jjjj)
#         writer.add_image(f'mask1_{xxxt}_{i}', vutils.make_grid(mask1_pyramid[0]),jjjj)
#         writer.add_image(f'mask2_{xxxt}_{i}', vutils.make_grid(mask2_pyramid[0]),jjjj)

        if i%200==0:
            print("6distrevisesdkssaving")
#             log("6distrevisedkssaving")
#             torch.save(model1_decoder.state_dict(), 'flow6fuk_distm1dev_epoch.pt')
#             torch.save(model1_encoder.state_dict(), 'flow6fuk_distm1en_epoch.pt')
            try:
                torch.save(model1.state_dict(), 'nomaskres18LeakATTm1.pt')
                torch.save(model2.state_dict(), 'nomaskres18LeakATTm2.pt')
                print("saved")
            except:
                print("sorry no memory")
#             writer.add_image(f'6expt{1}_v',  vutils.make_grid(before_tgt),xxxt)
#             writer.add_image(f'6exptdaepth_before{1}_v',  vutils.make_grid(pyramid_disp[0]),xxxt)
#             print("6_distrevisedkssaving finished") 
#             log("6_distreviecsedkssaving finished")
            
            results=eval(model1,test_data_loader)
            if results[0]<maxx:
                maxx=results[0]
                torch.save(model1.state_dict(), 'max_res18bestm1.pt')
            test(model1,device)
#             writer.add_scalar('abs', results[0],xxxt)
#             writer.add_scalar('sqr', results[1],xxxt)
#             writer.add_scalar('rms', results[2],xxxt)

    print(loss_epoch)
    v=0
#     for e in test_loader:
#         test_im=torch.tensor(e)/255
#         depth_test=model1(test_im.to(device))
    
#         writer.add_image(f'2expttest{v}_v',  vutils.make_grid(test_im),xxxt)
#         writer.add_image(f'2exptdaepth{v}_v',  vutils.make_grid(depth_test[0]),xxxt)
#         jjjj=jjjj+1
#         v=v+1
    log(loss_epoch)


saving
saved
6saving finished


/Data/tulikapcs16/anaconda3/envs/kaiser/lib/python3.7/site-packages/torch/nn/functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "
/Data/tulikapcs16/anaconda3/envs/kaiser/lib/python3.7/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


6distrevisesdkssaving
saved


























































    0.9250,    22.2607,     17.068,      0.846,     71.412,      0.228,      0.435,      0.596
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.8493,    19.9902,     16.636,      0.800,     70.772,      0.237,      0.447,      0.613
6distrevisesdkssaving
saved


























































    0.5388,     9.0813,     10.908,      0.544,     61.564,      0.352,      0.610,      0.801
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.4253,     5.7325,      9.162,      0.443,     54.731,      0.414,      0.725,      0.890
6distrevisesdkssaving
saved


























































    0.3768,     5.2769,     10.360, 

6distrevisesdkssaving
saved


























































    0.2923,     2.5538,      8.052,      0.377,     40.597,      0.521,      0.801,      0.916
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.2437,     1.8405,      6.466,      0.304,     35.292,      0.600,      0.868,      0.956
6distrevisesdkssaving
saved


























































    0.3119,     3.4463,      8.655,      0.385,     40.349,      0.538,      0.801,      0.908
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.2791,     3.3753,      8.114,      0.327,     36.511,      0.603,      0.852,      0.941
6distrevisesdkssaving
saved


























































    0.2938,     2.9081,      8.126, 

eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.2366,     2.5412,      7.325,      0.290,     31.736,      0.658,      0.889,      0.959
6distrevisesdkssaving
saved


























































    0.2605,     2.4095,      8.150,      0.361,     35.807,      0.570,      0.822,      0.917
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.2203,     1.6990,      6.648,      0.297,     32.267,      0.620,      0.875,      0.957
6distrevisesdkssaving
saved


























































    0.2761,     2.6510,      8.239,      0.371,     37.823,      0.547,      0.808,      0.916
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.2149,    

...

6distrevisesdkssaving
saved


























































    0.2690,     8.7121,      9.050,      0.318,     28.985,      0.717,      0.879,      0.947
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1488,     1.2903,      5.654,      0.207,     21.479,      0.814,      0.946,      0.982
6distrevisesdkssaving
saved


























































    0.2908,    19.4161,     10.916,      0.327,     28.548,      0.722,      0.881,      0.945
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1479,     1.2774,      5.595,      0.206,     21.323,      0.815,      0.947,      0.982
6distrevisesdkssaving
saved


























































    0.2954,    23.7048,     10.467, 

eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1548,     1.9512,      6.437,      0.212,     21.546,      0.812,      0.943,      0.980
6distrevisesdkssaving
saved


























































    0.2960,    26.5950,     11.240,      0.328,     28.960,      0.709,      0.879,      0.946
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1542,     1.3248,      5.734,      0.212,     21.895,      0.803,      0.944,      0.981
6distrevisesdkssaving
saved


























































    0.2788,    18.5971,     10.627,      0.322,     28.315,      0.725,      0.883,      0.945
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1544,    

6distrevisesdkssaving
saved


























































    0.2813,    14.4583,     10.255,      0.321,     27.904,      0.728,      0.884,      0.946
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1556,     1.5780,      6.090,      0.211,     21.697,      0.809,      0.944,      0.980
6distrevisesdkssaving
saved


























































    0.2885,    15.1398,     10.241,      0.325,     28.173,      0.718,      0.884,      0.946
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1610,     1.7829,      6.213,      0.216,     21.946,      0.805,      0.942,      0.979
6distrevisesdkssaving
saved


























































    0.2830,    17.5891,     10.680, 

eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1488,     1.5525,      6.040,      0.207,     21.087,      0.818,      0.946,      0.981
6distrevisesdkssaving
saved


























































    0.2897,    21.5078,     11.872,      0.323,     27.823,      0.731,      0.884,      0.946
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1475,     1.5311,      6.064,      0.204,     20.933,      0.821,      0.949,      0.982
6distrevisesdkssaving
saved


























































    0.2856,    16.7975,     10.486,      0.329,     28.862,      0.717,      0.878,      0.943
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1478,    


Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1512,     1.6377,      6.254,      0.208,     21.273,      0.816,      0.946,      0.981
tensor(1081.3682, device='cuda:0', grad_fn=<AddBackward0>)
saving
saved
6saving finished
6distrevisesdkssaving
saved


























































    0.2936,    25.9223,     11.171,      0.325,     28.545,      0.721,      0.878,      0.944
Total number of test images: 652

Evaluation results...
eigen_split
   abs_rel,     sq_rel,        rms,    log_rms,     d1_all,         a1,         a2,         a3
    0.1490,     1.3742,      5.784,      0.207,     21.237,      0.816,      0.947,      0.981
6distrevisesdkssaving
saved


























































    0.2928,    15.8096,     10.329,      0.329,     28.951,      0.704,      0.874,      0.942
Total number of test images: 652

Evaluation results...
eigen_spli

In [ ]:
# 0.1277,     0.9298,      5.341,      0.199,     19.522,      0.844,      0.956,      0.984
# 0.1270,     0.9074,      5.141,      0.194,     18.829,      0.850,      0.961,      0.985
# Res18ATTm1.pt is res50

In [24]:
3d-0.1444,     1.2184,      5.494,      0.200,     20.751,      0.827,      0.950,      0.983


Fri Jan 17 16:36:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:65:00.0 Off |                  N/A |
| 30%   47C    P8     6W / 250W |    164MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:B4:00.0 Off |                  N/A |
| 43%   71C    P2   106W / 250W |  10478MiB / 11019MiB |     33%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:
plt.imshow(warped_image1_pyramid[0][0].cpu().data.permute(1,2,0).data,cmap='gray')

In [ ]:
plt.imshow(warped_image1_pyramid[0][0].cpu().data.permute(1,2,0).data,cmap='gray')

In [ ]:
!nvidia-smi

In [ ]:
# xx=model1_decoder(model1_encoder(before_tgt))
model1=model1.to('cuda:3')
xx=model1(before_tgt.to('cuda:3'))

plt.imshow(before_tgt[1].permute(1,2,0).cpu().data)

In [ ]:
plt.imshow(before_tgt[2].permute(1,2,0).cpu().data)

In [ ]:
plt.imshow(xx[0][0][0].cpu().data,cmap='magma')


In [ ]:
plt.imshow(xx[0][2][0].cpu().data,cmap='gray')


In [ ]:
plt.imshow(mask2_pyramid[0][0][0].cpu().data,cmap='gray')


In [ ]:
plt.imshow(id_mp[0][0][0],cmap='gray')

In [ ]:
torch.zeros(4,1,256,512).sum(2).sum(2).shape